# Demo

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [11]:
class EpistasisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class EpistasisDetector(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EpistasisDetector, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels.unsqueeze(1)).sum().item()
    return correct / total

# generate synthetic genotype and phenotype data with epistatic interactions
X, y = make_classification(n_samples=1000, n_features=100, n_informative=10,
                           n_redundant=0, n_repeated=0, n_classes=2,
                           n_clusters_per_class=2, weights=None, flip_y=0.01,
                           class_sep=1.0, hypercube=True, shift=0.0,
                           scale=1.0, shuffle=True, random_state=42)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create PyTorch datasets
train_dataset = EpistasisDataset(X_train, y_train)
test_dataset = EpistasisDataset(X_test, y_test)

# create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# initialize the model, loss function, and optimizer
input_dim = X_train.shape[1]
hidden_dim = 64
output_dim = 1
model = EpistasisDetector(input_dim, hidden_dim, output_dim).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training loop
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    train_accuracy = evaluate(model, train_loader, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, "
          f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch [1/10], Train Loss: 0.6555, Train Accuracy: 0.7612, Test Accuracy: 0.6950
Epoch [2/10], Train Loss: 0.5474, Train Accuracy: 0.8462, Test Accuracy: 0.7350
Epoch [3/10], Train Loss: 0.4568, Train Accuracy: 0.8725, Test Accuracy: 0.7500
Epoch [4/10], Train Loss: 0.3798, Train Accuracy: 0.9000, Test Accuracy: 0.7800
Epoch [5/10], Train Loss: 0.3202, Train Accuracy: 0.9213, Test Accuracy: 0.8150
Epoch [6/10], Train Loss: 0.2744, Train Accuracy: 0.9325, Test Accuracy: 0.8400
Epoch [7/10], Train Loss: 0.2371, Train Accuracy: 0.9400, Test Accuracy: 0.8400
Epoch [8/10], Train Loss: 0.2061, Train Accuracy: 0.9525, Test Accuracy: 0.8550
Epoch [9/10], Train Loss: 0.1797, Train Accuracy: 0.9625, Test Accuracy: 0.8650
Epoch [10/10], Train Loss: 0.1569, Train Accuracy: 0.9688, Test Accuracy: 0.8700
